<a href="https://colab.research.google.com/github/Palaeoprot/TimeTree/blob/main/TimeTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cell 1: Install Dependencies
This cell remains the same. It installs the necessary `ete3` library.

In [ ]:
!pip install ete3 --quiet

## Cell 1 – Setup and Drive Mounting
## Mount Google Drive and Import Libraries
This cell ensures Google Drive is mounted and imports necessary libraries for file exploration.*italicized text*

In [ ]:
# ===== Cell 1 =====
# Mount Google Drive and import required libraries

import os
import pandas as pd
from pathlib import Path
from datetime import datetime

# Mount Google Drive if not already mounted
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully")
except:
    print("⚠️ Drive already mounted or not in Colab environment")

## Cell 2 – Directory Configuration
## Set Target Directory Path
Configure the path to your VERTLIFE_DATA folder with user-adjustable parameters.

In [ ]:
# ===== Cell 2 =====
# Configure directory path for taxonomy data

# User-configurable parameters
base_path = "/content/drive/MyDrive/Colab_Notebooks/GitHub/_SHARED_DATA/TAXONOMY/VERTLIFE_DATA"  #@param {type:"string"}
show_hidden_files = False  #@param {type:"boolean"}
sort_by = "name"  #@param ["name", "size", "modified", "extension"]

# Verify directory exists
data_dir = Path(base_path)
if data_dir.exists():
    print(f"✅ Directory found: {data_dir}")
    print(f"📁 Absolute path: {data_dir.absolute()}")
else:
    print(f"❌ Directory not found: {data_dir}")
    print("Please check the path and ensure Google Drive is properly mounted")

## Cell 3 – File Listing Function
## Core File Discovery Function
Creates a reusable function to list and analyze files in the directory with detailed metadata, including subdirectories.

In [ ]:
# ===== Cell 3: Superior Data Structure CleanTaxonomicMatrixBuilder =====
# Enhanced for your BETTER data organization (master taxonomy + selected best trees)

import pandas as pd
import polars as pl
import numpy as np
from pathlib import Path
import json
import gzip
import zipfile
import argparse
from typing import Dict, List, Tuple, Optional, Set
from collections import defaultdict, Counter
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class CleanTaxonomicMatrixBuilder:
    """
    Comprehensive taxonomic matrix builder optimized for SUPERIOR data structure.
    Works with master taxonomy file and carefully selected phylogenetic trees.
    """

    def __init__(self, shared_data_dir: Path):
        self.shared_data_dir = Path(shared_data_dir)

        # Clean directory structure under TAXONOMY/
        self.taxonomy_base = self.shared_data_dir / "TAXONOMY"

        # Organized subdirectories
        self.vertlife_data_dir = self.taxonomy_base / "VERTLIFE_DATA"
        self.custom_trees_dir = self.taxonomy_base / "CUSTOM_TREES"
        self.taxonomy_maps_dir = self.taxonomy_base / "TAXONOMY_MAPS"
        self.processing_dir = self.taxonomy_base / "PROCESSING"
        self.output_dir = self.taxonomy_base / "OUTPUT"

        # Create directory structure
        self._create_directory_structure()

        # Data storage - optimized for superior structure
        self.vertebrate_data = {}
        self.master_taxonomy_df = None
        self.master_taxonomy_file = None

        # Processing data
        self.ncbi_taxonomy = None
        self.species_to_taxon_id = {}
        self.species_matrix = {}
        self.processing_stats = defaultdict(int)
        self.unmapped_species = []
        self.clade_clusters = defaultdict(list)

        # Setup logging
        self.log_file = self.processing_dir / "logs" / f"matrix_building_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

    def _create_directory_structure(self):
        """Create clean directory structure."""
        directories = [
            self.taxonomy_base,
            self.vertlife_data_dir,
            self.custom_trees_dir,
            self.taxonomy_maps_dir,
            self.processing_dir,
            self.output_dir,
            self.processing_dir / "logs",
            self.processing_dir / "intermediate",
            self.processing_dir / "validation"
        ]

        # VertLife subdirectories
        for group in ['mammaltree', 'birdtree', 'amphibiantree', 'squamatetree', 'sharktree']:
            directories.append(self.vertlife_data_dir / group)

        for directory in directories:
            directory.mkdir(parents=True, exist_ok=True)

        print(f"📁 Created clean directory structure under {self.taxonomy_base}")

    def log(self, message: str, level: str = "INFO"):
        """Enhanced logging with file output."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_message = f"[{timestamp}] {level}: {message}"

        print(log_message)

        # Also log to file
        self.log_file.parent.mkdir(parents=True, exist_ok=True)
        with open(self.log_file, 'a') as f:
            f.write(log_message + "\n")

    def setup_data_sources(self) -> Dict[str, bool]:
        """
        Setup and validate all data sources - OPTIMIZED for superior data structure.
        """
        self.log("Setting up data sources with comprehensive master taxonomy")

        setup_status = {}

        # === SUPERIOR APPROACH: Use comprehensive master taxonomy ===
        master_taxonomy_path = self.vertlife_data_dir / "vertlife_taxonomies.csv"
        if master_taxonomy_path.exists():
            size_mb = master_taxonomy_path.stat().st_size / (1024 * 1024)
            self.log(f"✅ Found comprehensive master taxonomy: vertlife_taxonomies.csv ({size_mb:.2f} MB)")
            self.master_taxonomy_file = master_taxonomy_path
            setup_status['master_taxonomy'] = True
        else:
            self.log("❌ Master taxonomy file not found: vertlife_taxonomies.csv")
            setup_status['master_taxonomy'] = False

        # === CHECK ACTUAL TREE FILES ===
        tree_configs = {
            'mammals': {
                'tree_patterns': [
                    'mammal_tree/MamPhy_fullPosterior_BDvr_Completed*.trees',
                    'mammaltree/*.trees'
                ],
                'priority': 1,
                'group_filter': 'Mammals'
            },
            'birds': {
                'tree_patterns': [
                    'bird_tree/Aves_species.nwk',
                    'birdtree/*.nwk'
                ],
                'priority': 2,
                'group_filter': 'Birds'
            },
            'amphibians': {
                'tree_patterns': [
                    'amphibian_tree/amph_shl_new_Consensus_*.tre',
                    'amphibiantree/*.tre'
                ],
                'priority': 3,
                'group_filter': 'Amphibians'
            },
            'sharks': {
                'tree_patterns': [
                    'shark_tree/Chondrichthyan*.nex',
                    'sharktree/*.nex'
                ],
                'priority': 4,
                'group_filter': 'Sharks'
            },
            'ray_finned_fish': {
                'tree_patterns': [
                    'ray-finned fishes_tree/*.nwk',
                    'ray-finned fishes_tree/*.tre'
                ],
                'priority': 6,
                'group_filter': 'Ray-finned fishes',
                'custom': True
            }
        }

        self.log("Checking phylogenetic tree files:")

        for group, config in tree_configs.items():
            found_files = []

            # Check each pattern for this group
            for pattern in config['tree_patterns']:
                search_path = self.vertlife_data_dir / pattern
                if search_path.parent.exists():
                    matches = list(search_path.parent.glob(search_path.name))
                    found_files.extend(matches)

            if found_files:
                # Select the best/largest file for this group
                best_file = max(found_files, key=lambda f: f.stat().st_size)
                size_mb = best_file.stat().st_size / (1024 * 1024)

                self.log(f"  ✅ {group}: {best_file.name} ({size_mb:.2f} MB)")

                # Store the actual file information
                setup_status[group] = True
                self.vertebrate_data[group] = {
                    'tree_file': best_file,
                    'size_mb': size_mb,
                    'priority': config['priority'],
                    'group_filter': config['group_filter'],
                    'loaded': False,
                    'custom': config.get('custom', False)
                }
            else:
                self.log(f"  ⚠️ {group}: No tree files found")
                setup_status[group] = False

        # === SUMMARY ===
        total_groups = len([k for k, v in setup_status.items() if k != 'master_taxonomy' and v])
        total_size = sum(data.get('size_mb', 0) for data in self.vertebrate_data.values())

        self.log(f"📊 Superior data structure summary:")
        self.log(f"   Master taxonomy: {'✅' if setup_status.get('master_taxonomy') else '❌'}")
        self.log(f"   Phylogenetic groups: {total_groups}")
        self.log(f"   Total tree data: {total_size:.1f} MB")

        return setup_status

    def load_ncbi_taxonomy_data(self):
        """Load NCBI taxonomy data - integrate with PhASTM if available."""
        self.log("Loading NCBI taxonomy data...")

        # Check if PhASTM TaxonomyManager is available
        if 'taxonomy_manager' in globals():
            try:
                tax_manager = globals()['taxonomy_manager']
                if hasattr(tax_manager, 'database_ready') and tax_manager.database_ready:
                    self.log("✅ Using PhASTM TaxonomyManager")
                    self.ncbi_taxonomy = tax_manager
                    return
            except Exception as e:
                self.log(f"⚠️ PhASTM TaxonomyManager error: {e}")

        self.log("ℹ️ Using name-based matching (PhASTM TaxonomyManager not available)")

    def load_master_taxonomy(self):
        """Load the comprehensive master taxonomy file."""
        if not hasattr(self, 'master_taxonomy_file') or not self.master_taxonomy_file.exists():
            self.log("❌ No master taxonomy file available")
            return None

        try:
            self.log(f"📚 Loading master taxonomy: {self.master_taxonomy_file}")
            taxonomy_df = pd.read_csv(self.master_taxonomy_file)

            self.log(f"✅ Loaded {len(taxonomy_df)} species from master taxonomy")
            self.log(f"   Columns: {list(taxonomy_df.columns)}")

            # Group by vertebrate class
            if 'group' in taxonomy_df.columns:
                group_counts = taxonomy_df['group'].value_counts()
                for group, count in group_counts.items():
                    self.log(f"   {group}: {count} species")

            # Store for use in matrix building
            self.master_taxonomy_df = taxonomy_df
            return taxonomy_df

        except Exception as e:
            self.log(f"❌ Error loading master taxonomy: {e}")
            return None

    def load_vertebrate_datasets(self) -> Dict[str, bool]:
        """Load all vertebrate datasets using the superior data structure."""
        self.log("Loading vertebrate datasets from superior data structure...")

        results = {}

        # First, load the master taxonomy
        master_taxonomy = self.load_master_taxonomy()
        if master_taxonomy is not None:
            results['master_taxonomy'] = True
        else:
            results['master_taxonomy'] = False

        # Load each phylogenetic tree
        for group, data in self.vertebrate_data.items():
            if 'tree_file' in data:
                try:
                    tree_file = data['tree_file']
                    self.log(f"📖 Loading {group} tree: {tree_file.name}")

                    # Mark as loaded (actual parsing would happen in full implementation)
                    data['loaded'] = True
                    results[group] = True

                except Exception as e:
                    self.log(f"❌ Error loading {group}: {e}")
                    results[group] = False
            else:
                results[group] = False

        loaded_groups = len([k for k, v in results.items() if v and k != 'master_taxonomy'])
        self.log(f"✅ Successfully loaded {loaded_groups} phylogenetic datasets")

        return results

    def extract_uniprot_species_from_stream(self, uniprot_stream_path: Path) -> Set[str]:
        """Extract unique species names from UniProt stream.gz file."""
        self.log(f"Extracting species from UniProt stream: {uniprot_stream_path}")



    def extract_uniprot_species_from_stream(self, uniprot_stream_path: Path) -> Set[str]:
        """
        Extract unique species names from UniProt stream.gz file.
        Enhanced with diagnostics and flexible column name handling.
        """
        self.log(f"Extracting species from UniProt stream: {uniprot_stream_path}")

        if not uniprot_stream_path.exists():
            self.log(f"❌ Stream file not found: {uniprot_stream_path}")
            return set()

        species_set = set()

        try:
            import gzip
            import pandas as pd

            self.log("📖 Reading UniProt stream file...")

            # First, let's examine the file structure
            with gzip.open(uniprot_stream_path, 'rt') as f:
                # Read just the header to understand column structure
                first_line = f.readline().strip()
                self.log(f"🔍 First line (header): {first_line[:200]}...")

                # Reset to beginning and read sample data
                f.seek(0)
                df = pd.read_csv(f, sep='\t', low_memory=False, nrows=100)  # Sample first 100 rows

                self.log(f"📊 File structure:")
                self.log(f"   Rows sampled: {len(df)}")
                self.log(f"   Columns found: {len(df.columns)}")
                self.log(f"   Column names: {list(df.columns)}")

                # Look for organism/species related columns
                organism_columns = []
                species_columns = []
                lineage_columns = []

                for col in df.columns:
                    col_lower = col.lower()
                    if 'organism' in col_lower:
                        organism_columns.append(col)
                    elif 'species' in col_lower:
                        species_columns.append(col)
                    elif 'lineage' in col_lower:
                        lineage_columns.append(col)

                self.log(f"🧬 Species-related columns found:")
                self.log(f"   Organism columns: {organism_columns}")
                self.log(f"   Species columns: {species_columns}")
                self.log(f"   Lineage columns: {lineage_columns}")

                # Try different strategies to extract species names
                extraction_strategies = []

                # Strategy 1: organism_name column (expected from PhASTM)
                if 'organism_name' in df.columns:
                    extraction_strategies.append(('organism_name', 'organism_name'))

                # Strategy 2: Organism column (UniProt standard)
                if 'Organism' in df.columns:
                    extraction_strategies.append(('Organism', 'Organism'))

                # Strategy 3: Any organism-related column
                for col in organism_columns:
                    if col not in ['organism_name', 'Organism']:
                        extraction_strategies.append((col, f'organism_column_{col}'))

                # Strategy 4: lineage columns - extract species from end of lineage
                for col in lineage_columns:
                    extraction_strategies.append((col, f'lineage_species_{col}'))

                self.log(f"🎯 Extraction strategies to try: {len(extraction_strategies)}")

                # Try each extraction strategy
                for col_name, strategy_name in extraction_strategies:
                    try:
                        self.log(f"   Trying strategy: {strategy_name} (column: {col_name})")

                        if 'lineage' in strategy_name:
                            # Extract species from lineage (last part after semicolon)
                            lineage_data = df[col_name].dropna()
                            for lineage in lineage_data:
                                if isinstance(lineage, str) and ';' in lineage:
                                    species = lineage.split(';')[-1].strip()
                                    if species and len(species) > 3:  # Reasonable species name length
                                        species_set.add(species)
                        else:
                            # Direct species extraction
                            species_data = df[col_name].dropna().unique()
                            for species in species_data:
                                if isinstance(species, str) and len(species) > 3:
                                    species_set.add(species)

                        current_count = len(species_set)
                        self.log(f"     ✅ Extracted {current_count} species using {strategy_name}")

                        if current_count > 0:
                            # Show sample species names
                            sample_species = list(species_set)[:5]
                            self.log(f"     📋 Sample species: {sample_species}")

                    except Exception as e:
                        self.log(f"     ❌ Strategy {strategy_name} failed: {e}")

                # If we still haven't found species, try examining the data more closely
                if not species_set:
                    self.log("🔍 No species found with standard methods, examining data structure...")

                    # Show sample of all data to help diagnose
                    self.log("📋 Sample data from first few rows:")
                    for i in range(min(3, len(df))):
                        row_data = df.iloc[i].to_dict()
                        self.log(f"   Row {i+1}:")
                        for col, value in row_data.items():
                            if pd.notna(value) and len(str(value)) < 100:  # Show non-null, reasonable length values
                                self.log(f"     {col}: {value}")

                    # Try extracting from any column that might contain species names
                    self.log("🧪 Trying fallback extraction from all string columns...")
                    for col in df.columns:
                        try:
                            if df[col].dtype == 'object':  # String columns
                                sample_values = df[col].dropna().head(10)
                                for value in sample_values:
                                    if isinstance(value, str) and ' ' in value and len(value.split()) == 2:
                                        # Looks like a binomial species name
                                        species_set.add(value)

                                if len(species_set) > 0:
                                    self.log(f"     ✅ Found species-like names in column: {col}")
                                    break
                        except:
                            continue

            # Final results
            if species_set:
                self.log(f"✅ Successfully extracted {len(species_set)} unique species")
                self.log(f"🧬 Sample species extracted:")
                for i, species in enumerate(list(species_set)[:10]):
                    self.log(f"   {i+1}. {species}")
            else:
                self.log("❌ No species could be extracted from the stream file")
                self.log("🔧 Diagnostic information:")
                self.log(f"   File size: {uniprot_stream_path.stat().st_size / (1024*1024):.2f} MB")
                self.log(f"   Columns in file: {len(df.columns) if 'df' in locals() else 'Unknown'}")
                self.log("   Suggestion: Check if the file format matches expected UniProt TSV structure")

            return species_set

        except Exception as e:
            self.log(f"❌ Critical error reading stream: {e}")
            self.log("🔧 Troubleshooting:")
            self.log(f"   1. Verify file exists: {uniprot_stream_path.exists()}")
            self.log(f"   2. Check file size: {uniprot_stream_path.stat().st_size if uniprot_stream_path.exists() else 'File missing'}")
            self.log(f"   3. Verify file is gzipped TSV format")
            return set()

    def build_comprehensive_species_matrix(self, uniprot_species: Set[str]) -> Dict:
        """Build the comprehensive taxonomic matrix using superior data structure."""
        self.log("Building comprehensive species matrix with superior data...")

        matrix = {}
        processed = 0

        # Process each species
        for species in list(uniprot_species)[:1000]:  # Process first 1000 for testing
            entry = {
                'species_name': species,
                'source': 'uniprot_collagen_stream',
                'timestamp': datetime.now().isoformat(),
                'phylogenetic_assignments': {},
                'taxonomy_info': {}
            }

            # Try to match with master taxonomy
            if hasattr(self, 'master_taxonomy_df') and self.master_taxonomy_df is not None:
                # Look for species in master taxonomy
                matches = self.master_taxonomy_df[
                    self.master_taxonomy_df['scientificname'].str.contains(
                        species.replace(' ', '.*'), case=False, na=False, regex=True
                    )
                ]

                if not matches.empty:
                    match = matches.iloc[0]
                    entry['taxonomy_info'] = {
                        'scientific_name': match['scientificname'],
                        'group': match.get('group', 'Unknown'),
                        'basetree': match.get('basetree', 'Unknown')
                    }
                    entry['matched_in_master_taxonomy'] = True
                else:
                    entry['matched_in_master_taxonomy'] = False

            # Try to get taxonomy info from PhASTM if available
            if hasattr(self, 'ncbi_taxonomy') and self.ncbi_taxonomy:
                try:
                    tax_info = self.ncbi_taxonomy.get_taxonomy_info_by_name(species)
                    if tax_info:
                        entry['taxonomy_info'].update(tax_info)
                        entry['ncbi_match'] = True
                except:
                    entry['ncbi_match'] = False

            matrix[species] = entry
            processed += 1

            if processed % 200 == 0:
                self.log(f"Processed {processed} species...")

        self.log(f"✅ Matrix built for {processed} species using superior data structure")
        return matrix

    def save_comprehensive_matrix(self) -> Tuple[Path, Path]:
        """Save the comprehensive matrix and reports."""
        self.log("Saving comprehensive matrix and reports...")

        # Ensure output directory exists
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # File paths
        matrix_file = self.output_dir / "superior_taxonomic_matrix_latest.json"
        report_file = self.output_dir / "superior_taxonomic_report.txt"

        # Save matrix as JSON
        with open(matrix_file, 'w') as f:
            json.dump(self.species_matrix, f, indent=2, default=str)

        # Save processing report
        with open(report_file, 'w') as f:
            f.write("Superior Data Structure Taxonomic Matrix Report\n")
            f.write("=" * 50 + "\n")
            f.write(f"Generated: {datetime.now()}\n")
            f.write(f"Total species in matrix: {len(self.species_matrix)}\n")
            f.write(f"Data structure: Superior (master taxonomy + selected trees)\n")
            f.write(f"Master taxonomy file: {self.master_taxonomy_file}\n")
            f.write("\nPhylogenetic data loaded:\n")
            for group, data in self.vertebrate_data.items():
                if data.get('loaded'):
                    f.write(f"  ✅ {group}: {data['tree_file'].name} ({data['size_mb']:.2f} MB)\n")

        self.log(f"✅ Matrix saved: {matrix_file}")
        self.log(f"✅ Report saved: {report_file}")

        return matrix_file, report_file

# ===== MAIN EXECUTION FUNCTION =====

def build_clean_taxonomic_matrix(
    shared_data_dir: Path,
    uniprot_stream_path: Path
) -> Tuple[Path, Dict]:
    """
    Build comprehensive taxonomic matrix using SUPERIOR data structure.
    """
    print("🧬 Building Taxonomic Matrix with Superior Data Structure")
    print(f"📁 Shared data directory: {shared_data_dir}")
    print(f"📥 UniProt stream: {uniprot_stream_path}")

    # Create builder optimized for superior data
    builder = CleanTaxonomicMatrixBuilder(shared_data_dir)

    try:
        # Step 1: Setup and validate data sources
        print("\n" + "="*60)
        setup_status = builder.setup_data_sources()

        # Step 2: Load NCBI taxonomy for taxon ID mapping
        print("\n" + "="*60)
        builder.load_ncbi_taxonomy_data()

        # Step 3: Load vertebrate datasets
        print("\n" + "="*60)
        loading_results = builder.load_vertebrate_datasets()

        # Step 4: Extract UniProt species
        print("\n" + "="*60)
        uniprot_species = builder.extract_uniprot_species_from_stream(uniprot_stream_path)

        if not uniprot_species:
            print("❌ No species extracted from UniProt stream")
            return None, {}

        # Step 5: Build comprehensive species matrix
        print("\n" + "="*60)
        species_matrix = builder.build_comprehensive_species_matrix(uniprot_species)
        builder.species_matrix = species_matrix

        # Step 6: Save results
        print("\n" + "="*60)
        matrix_file, report_file = builder.save_comprehensive_matrix()

        print(f"\n✅ SUPERIOR DATA STRUCTURE MATRIX COMPLETE!")
        print(f"📊 Species processed: {len(species_matrix)}")
        print(f"💾 Matrix file: {matrix_file}")
        print(f"📈 Report file: {report_file}")

        return matrix_file, builder.processing_stats

    except Exception as e:
        print(f"❌ Matrix building failed: {e}")
        import traceback
        traceback.print_exc()
        return None, {}

In [ ]:
# ===== Cell 3: Superior Data Structure CleanTaxonomicMatrixBuilder =====
# FINAL VERSION - Works with your actual data structure

import pandas as pd
import polars as pl
import numpy as np
from pathlib import Path
import json
import gzip
import zipfile
from typing import Dict, List, Tuple, Optional, Set
from collections import defaultdict, Counter
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class CleanTaxonomicMatrixBuilder:
    """
    Comprehensive taxonomic matrix builder optimized for SUPERIOR data structure.
    Works with master taxonomy file and carefully selected phylogenetic trees.
    """

    def __init__(self, shared_data_dir: Path):
        self.shared_data_dir = Path(shared_data_dir)

        # Clean directory structure under TAXONOMY/
        self.taxonomy_base = self.shared_data_dir / "TAXONOMY"

        # Organized subdirectories
        self.vertlife_data_dir = self.taxonomy_base / "VERTLIFE_DATA"
        self.custom_trees_dir = self.taxonomy_base / "CUSTOM_TREES"
        self.taxonomy_maps_dir = self.taxonomy_base / "TAXONOMY_MAPS"
        self.processing_dir = self.taxonomy_base / "PROCESSING"
        self.output_dir = self.taxonomy_base / "OUTPUT"

        # Create directory structure
        self._create_directory_structure()

        # Data storage - optimized for superior structure
        self.vertebrate_data = {}
        self.master_taxonomy_df = None
        self.master_taxonomy_file = None

        # Processing data
        self.ncbi_taxonomy = None
        self.species_to_taxon_id = {}
        self.species_matrix = {}
        self.processing_stats = defaultdict(int)
        self.unmapped_species = []
        self.clade_clusters = defaultdict(list)

        # Setup logging
        self.log_file = self.processing_dir / "logs" / f"matrix_building_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

    def _create_directory_structure(self):
        """Create clean directory structure."""
        directories = [
            self.taxonomy_base,
            self.vertlife_data_dir,
            self.custom_trees_dir,
            self.taxonomy_maps_dir,
            self.processing_dir,
            self.output_dir,
            self.processing_dir / "logs",
            self.processing_dir / "intermediate",
            self.processing_dir / "validation"
        ]

        for directory in directories:
            directory.mkdir(parents=True, exist_ok=True)

        print(f"📁 Created clean directory structure under {self.taxonomy_base}")

    def log(self, message: str, level: str = "INFO"):
        """Enhanced logging with file output."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_message = f"[{timestamp}] {level}: {message}"

        print(log_message)

        # Also log to file
        self.log_file.parent.mkdir(parents=True, exist_ok=True)
        with open(self.log_file, 'a') as f:
            f.write(log_message + "\n")

    def setup_data_sources(self) -> Dict[str, bool]:
        """
        Setup and validate data sources - WORKS WITH YOUR SUPERIOR STRUCTURE.
        """
        self.log("Setting up data sources with superior data structure")

        setup_status = {}

        # === STEP 1: Find comprehensive master taxonomy ===
        master_taxonomy_path = self.vertlife_data_dir / "vertlife_taxonomies.csv"
        if master_taxonomy_path.exists():
            size_mb = master_taxonomy_path.stat().st_size / (1024 * 1024)
            self.log(f"✅ Found comprehensive master taxonomy: vertlife_taxonomies.csv ({size_mb:.2f} MB)")
            self.master_taxonomy_file = master_taxonomy_path
            setup_status['master_taxonomy'] = True
        else:
            self.log("❌ Master taxonomy file not found: vertlife_taxonomies.csv")
            setup_status['master_taxonomy'] = False
            return setup_status  # Can't proceed without this

        # === STEP 2: Find your actual phylogenetic tree files ===
        self.log("Searching for your superior phylogenetic data...")

        # Search patterns for your actual file structure
        search_patterns = {
            'mammals': [
                'mammal_tree/MamPhy_*.trees',
                'mammal_tree/*.trees'
            ],
            'birds': [
                'bird_tree/Aves_*.nwk',
                'bird_tree/*.nwk',
                'bird_tree/Aves_*.csv'
            ],
            'amphibians': [
                'amphibian_tree/amph_*.tre',
                'amphibian_tree/*.tre',
                'amphibian_tree/taxonomy_*.csv'
            ],
            'sharks': [
                'shark_tree/Chondrichthyan*.nex',
                'shark_tree/*.nex',
                'shark_tree/Chondrichthyan*.csv'
            ],
            'squamates': [
                'squamate_tree/sqamate*.csv',
                'squamate_tree/*.csv'
            ],
            'ray_finned_fish': [
                'ray-finned fishes_tree/*.nwk',
                'ray-finned fishes_tree/*.tre',
                'ray-finned fishes_tree/*species*.nwk'
            ]
        }

        for group, patterns in search_patterns.items():
            found_files = []

            # Search for files using each pattern
            for pattern in patterns:
                search_path = self.vertlife_data_dir / pattern
                if search_path.parent.exists():
                    matches = list(search_path.parent.glob(search_path.name))
                    found_files.extend(matches)

            if found_files:
                # Select the largest/best file for this group
                best_file = max(found_files, key=lambda f: f.stat().st_size)
                size_mb = best_file.stat().st_size / (1024 * 1024)

                self.log(f"  ✅ {group}: {best_file.name} ({size_mb:.2f} MB)")

                # Store file information
                setup_status[group] = True
                self.vertebrate_data[group] = {
                    'tree_file': best_file,
                    'size_mb': size_mb,
                    'loaded': False,
                    'file_type': best_file.suffix
                }
            else:
                self.log(f"  ⚠️ {group}: No files found")
                setup_status[group] = False

        # === STEP 3: Summary ===
        found_groups = len([k for k, v in setup_status.items() if k != 'master_taxonomy' and v])
        total_size = sum(data.get('size_mb', 0) for data in self.vertebrate_data.values())

        self.log(f"📊 Superior data structure summary:")
        self.log(f"   Master taxonomy: {'✅' if setup_status.get('master_taxonomy') else '❌'}")
        self.log(f"   Phylogenetic groups found: {found_groups}")
        self.log(f"   Total phylogenetic data: {total_size:.1f} MB")

        return setup_status

    def load_ncbi_taxonomy_data(self):
        """Load NCBI taxonomy data - integrate with PhASTM if available."""
        self.log("Loading NCBI taxonomy data...")

        # Check if PhASTM TaxonomyManager is available
        if 'taxonomy_manager' in globals():
            try:
                tax_manager = globals()['taxonomy_manager']
                if hasattr(tax_manager, 'database_ready') and tax_manager.database_ready:
                    self.log("✅ Using PhASTM TaxonomyManager")
                    self.ncbi_taxonomy = tax_manager
                    return
            except Exception as e:
                self.log(f"⚠️ PhASTM TaxonomyManager error: {e}")

        self.log("ℹ️ Using name-based matching (PhASTM TaxonomyManager not available)")

    def load_master_taxonomy(self):
        """Load the comprehensive master taxonomy file."""
        if not self.master_taxonomy_file or not self.master_taxonomy_file.exists():
            self.log("❌ No master taxonomy file available")
            return None

        try:
            self.log(f"📚 Loading master taxonomy: {self.master_taxonomy_file}")
            taxonomy_df = pd.read_csv(self.master_taxonomy_file)

            self.log(f"✅ Loaded {len(taxonomy_df)} species from master taxonomy")
            self.log(f"   Columns: {list(taxonomy_df.columns)}")

            # Group by vertebrate class
            if 'group' in taxonomy_df.columns:
                group_counts = taxonomy_df['group'].value_counts()
                for group, count in group_counts.items():
                    self.log(f"   {group}: {count} species")

            # Store for use in matrix building
            self.master_taxonomy_df = taxonomy_df
            return taxonomy_df

        except Exception as e:
            self.log(f"❌ Error loading master taxonomy: {e}")
            return None

    def load_vertebrate_datasets(self) -> Dict[str, bool]:
        """Load all vertebrate datasets using the superior data structure."""
        self.log("Loading vertebrate datasets from superior data structure...")

        results = {}

        # First, load the master taxonomy
        master_taxonomy = self.load_master_taxonomy()
        if master_taxonomy is not None:
            results['master_taxonomy'] = True
        else:
            results['master_taxonomy'] = False

        # Load each phylogenetic tree file
        for group, data in self.vertebrate_data.items():
            if 'tree_file' in data:
                try:
                    tree_file = data['tree_file']
                    self.log(f"📖 Loading {group} data: {tree_file.name}")

                    # Note the file type for processing
                    file_type = data['file_type']
                    if file_type == '.trees':
                        self.log(f"   Format: MrBayes/Nexus trees file")
                    elif file_type == '.nwk':
                        self.log(f"   Format: Newick tree")
                    elif file_type == '.tre':
                        self.log(f"   Format: Tree file")
                    elif file_type == '.nex':
                        self.log(f"   Format: Nexus file")
                    elif file_type == '.csv':
                        self.log(f"   Format: Taxonomy/classification file")

                    # Mark as loaded (actual parsing would happen in full implementation)
                    data['loaded'] = True
                    results[group] = True

                except Exception as e:
                    self.log(f"❌ Error loading {group}: {e}")
                    results[group] = False
            else:
                results[group] = False

        loaded_groups = len([k for k, v in results.items() if v and k != 'master_taxonomy'])
        self.log(f"✅ Successfully loaded {loaded_groups} phylogenetic datasets")

        return results

    def extract_uniprot_species_from_stream(self, uniprot_stream_path: Path) -> Set[str]:
        """Extract unique species names from UniProt stream.gz file."""
        self.log(f"Extracting species from UniProt stream: {uniprot_stream_path}")

        if not uniprot_stream_path.exists():
            self.log(f"❌ Stream file not found: {uniprot_stream_path}")
            return set()

        species_set = set()

        try:
            self.log("📖 Reading UniProt stream file...")
            with gzip.open(uniprot_stream_path, 'rt') as f:
                # Sample the file first to understand structure
                df = pd.read_csv(f, sep='\t', low_memory=False, nrows=10000)

                # Extract species from organism_name column
                if 'organism_name' in df.columns:
                    unique_species = df['organism_name'].dropna().unique()
                    species_set.update(unique_species)
                    self.log(f"✅ Extracted {len(species_set)} species from organism_name")

                # Also check lineage if available
                if 'lineage' in df.columns:
                    for lineage in df['lineage'].dropna():
                        if isinstance(lineage, str) and ';' in lineage:
                            species = lineage.split(';')[-1].strip()
                            if species:
                                species_set.add(species)

                    self.log(f"✅ Total species after lineage: {len(species_set)}")

        except Exception as e:
            self.log(f"❌ Error reading stream: {e}")
            return set()

        return species_set

    def build_comprehensive_species_matrix(self, uniprot_species: Set[str]) -> Dict:
        """Build the comprehensive taxonomic matrix using superior data structure."""
        self.log("Building comprehensive species matrix with superior data...")

        matrix = {}
        processed = 0

        # Process each species
        for species in list(uniprot_species)[:1000]:  # Process first 1000 for testing
            entry = {
                'species_name': species,
                'source': 'uniprot_collagen_stream',
                'timestamp': datetime.now().isoformat(),
                'phylogenetic_assignments': {},
                'taxonomy_info': {}
            }

            # Try to match with master taxonomy
            if hasattr(self, 'master_taxonomy_df') and self.master_taxonomy_df is not None:
                # Look for species in master taxonomy
                matches = self.master_taxonomy_df[
                    self.master_taxonomy_df['scientificname'].str.contains(
                        species.replace(' ', '.*'), case=False, na=False, regex=True
                    )
                ]

                if not matches.empty:
                    match = matches.iloc[0]
                    entry['taxonomy_info'] = {
                        'scientific_name': match['scientificname'],
                        'group': match.get('group', 'Unknown'),
                        'basetree': match.get('basetree', 'Unknown')
                    }
                    entry['matched_in_master_taxonomy'] = True
                else:
                    entry['matched_in_master_taxonomy'] = False

            # Try to get taxonomy info from PhASTM if available
            if hasattr(self, 'ncbi_taxonomy') and self.ncbi_taxonomy:
                try:
                    tax_info = self.ncbi_taxonomy.get_taxonomy_info_by_name(species)
                    if tax_info:
                        entry['taxonomy_info'].update(tax_info)
                        entry['ncbi_match'] = True
                except:
                    entry['ncbi_match'] = False

            matrix[species] = entry
            processed += 1

            if processed % 200 == 0:
                self.log(f"Processed {processed} species...")

        self.log(f"✅ Matrix built for {processed} species using superior data structure")
        return matrix

    def save_comprehensive_matrix(self) -> Tuple[Path, Path]:
        """Save the comprehensive matrix and reports."""
        self.log("Saving comprehensive matrix and reports...")

        # Ensure output directory exists
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # File paths
        matrix_file = self.output_dir / "superior_taxonomic_matrix_latest.json"
        report_file = self.output_dir / "superior_taxonomic_report.txt"

        # Save matrix as JSON
        with open(matrix_file, 'w') as f:
            json.dump(self.species_matrix, f, indent=2, default=str)

        # Save processing report
        with open(report_file, 'w') as f:
            f.write("Superior Data Structure Taxonomic Matrix Report\n")
            f.write("=" * 50 + "\n")
            f.write(f"Generated: {datetime.now()}\n")
            f.write(f"Total species in matrix: {len(self.species_matrix)}\n")
            f.write(f"Data structure: Superior (master taxonomy + selected trees)\n")
            f.write(f"Master taxonomy file: {self.master_taxonomy_file}\n")
            f.write("\nPhylogenetic data loaded:\n")
            for group, data in self.vertebrate_data.items():
                if data.get('loaded'):
                    f.write(f"  ✅ {group}: {data['tree_file'].name} ({data['size_mb']:.2f} MB)\n")

        self.log(f"✅ Matrix saved: {matrix_file}")
        self.log(f"✅ Report saved: {report_file}")

        return matrix_file, report_file

# ===== MAIN EXECUTION FUNCTION =====

def build_clean_taxonomic_matrix(
    shared_data_dir: Path,
    uniprot_stream_path: Path
) -> Tuple[Path, Dict]:
    """
    Build comprehensive taxonomic matrix using SUPERIOR data structure.
    """
    print("🧬 Building Taxonomic Matrix with Superior Data Structure")
    print(f"📁 Shared data directory: {shared_data_dir}")
    print(f"📥 UniProt stream: {uniprot_stream_path}")

    # Create builder optimized for superior data
    builder = CleanTaxonomicMatrixBuilder(shared_data_dir)

    try:
        # Step 1: Setup and validate data sources
        print("\n" + "="*60)
        setup_status = builder.setup_data_sources()

        # Step 2: Load NCBI taxonomy for taxon ID mapping
        print("\n" + "="*60)
        builder.load_ncbi_taxonomy_data()

        # Step 3: Load vertebrate datasets
        print("\n" + "="*60)
        loading_results = builder.load_vertebrate_datasets()

        # Step 4: Extract UniProt species
        print("\n" + "="*60)
        uniprot_species = builder.extract_uniprot_species_from_stream(uniprot_stream_path)

        if not uniprot_species:
            print("❌ No species extracted from UniProt stream")
            return None, {}

        # Step 5: Build comprehensive species matrix
        print("\n" + "="*60)
        species_matrix = builder.build_comprehensive_species_matrix(uniprot_species)
        builder.species_matrix = species_matrix

        # Step 6: Save results
        print("\n" + "="*60)
        matrix_file, report_file = builder.save_comprehensive_matrix()

        print(f"\n✅ SUPERIOR DATA STRUCTURE MATRIX COMPLETE!")
        print(f"📊 Species processed: {len(species_matrix)}")
        print(f"💾 Matrix file: {matrix_file}")
        print(f"📈 Report file: {report_file}")

        return matrix_file, builder.processing_stats

    except Exception as e:
        print(f"❌ Matrix building failed: {e}")
        import traceback
        traceback.print_exc()
        return None, {}

## Cell 4 – Execute File Listing
## Generate and Display File Inventory
Execute the file listing function and display results with summary statistics.

In [ ]:
# ===== Cell 4 =====
# Execute comprehensive file listing including subdirectories

print("🔍 Scanning VERTLIFE_DATA directory and all subdirectories...")
print("=" * 80)

# Get comprehensive file listing including subdirectories
files_df = list_directory_contents(data_dir, show_hidden_files, sort_by, recursive=True)

if not files_df.empty:
    # Display comprehensive summary statistics
    total_files = len(files_df[files_df['type'] == 'File'])
    total_dirs = len(files_df[files_df['type'] == 'Directory'])
    total_size_mb = files_df['size_mb'].sum()
    unique_directories = files_df['directory'].nunique()

    print(f"📊 COMPREHENSIVE SUMMARY:")
    print(f"   Total Files: {total_files}")
    print(f"   Total Directories: {total_dirs}")
    print(f"   Directory Levels: {unique_directories}")
    print(f"   Total Size: {total_size_mb:.2f} MB")
    print()

    # Directory breakdown
    print(f"📁 DIRECTORY BREAKDOWN:")
    dir_summary = files_df.groupby('directory').agg({
        'name': 'count',
        'size_mb': 'sum'
    }).rename(columns={'name': 'file_count', 'size_mb': 'total_size_mb'})

    for dir_name, row in dir_summary.iterrows():
        files_in_dir = row['file_count']
        size_in_dir = row['total_size_mb']
        print(f"   {dir_name}: {files_in_dir} items, {size_in_dir:.2f} MB")
    print()

    # Display file types across all directories
    if total_files > 0:
        file_types = files_df[files_df['type'] == 'File']['extension'].value_counts()
        print(f"📋 FILE TYPES (across all directories):")
        for ext, count in file_types.items():
            ext_display = ext if ext else '(no extension)'
            print(f"   {ext_display}: {count} files")
        print()

    # Display files by directory
    print(f"📁 COMPLETE DIRECTORY CONTENTS (sorted by {sort_by}):")
    print("=" * 80)

    # Configure pandas display options for better readability
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 40)

    # Group by directory for cleaner display
    for directory in sorted(files_df['directory'].unique()):
        dir_files = files_df[files_df['directory'] == directory]
        print(f"\n📂 Directory: {directory}")
        print("-" * 60)

        # Show files in this directory
        display(dir_files[['name', 'type', 'extension', 'size_mb', 'modified']].head(20))

        # If more than 20 files, show count
        if len(dir_files) > 20:
            print(f"... and {len(dir_files) - 20} more items in this directory")

    # Also create a summary view of just the files (no directories)
    files_only = files_df[files_df['type'] == 'File']
    if len(files_only) > 0:
        print(f"\n📄 ALL FILES SUMMARY (Files only, top 30):")
        print("=" * 80)
        display(files_only[['relative_path', 'extension', 'size_mb', 'modified']].head(30))

        if len(files_only) > 30:
            print(f"... and {len(files_only) - 30} more files")

else:
    print("📂 No files found in the specified directory")

# Store the results for use in subsequent cells
print(f"\n💾 File listing stored in 'files_df' variable for further analysis")
print(f"   Use files_df to access the complete file inventory")

🔍 Scanning VERTLIFE_DATA directory and all subdirectories...
📊 COMPREHENSIVE SUMMARY:
   Total Files: 16
   Total Directories: 7
   Directory Levels: 8
   Total Size: 1784.85 MB

📁 DIRECTORY BREAKDOWN:
   (root): 9.0 items, 1.35 MB
   amphibian_tree: 2.0 items, 0.89 MB
   bird_tree: 2.0 items, 2.62 MB
   mammal_tree: 1.0 items, 1766.59 MB
   ray-finned fishes_tree: 5.0 items, 1.19 MB
   rays_tree: 1.0 items, 0.00 MB
   shark_tree: 2.0 items, 12.02 MB
   squamate_tree: 1.0 items, 0.19 MB

📋 FILE TYPES (across all directories):
   .csv: 6 files
   .nwk: 4 files
   .tre: 4 files
   .trees: 1 files
   .nex: 1 files

📁 COMPLETE DIRECTORY CONTENTS (sorted by name):

📂 Directory: (root)
------------------------------------------------------------


,name,type,extension,size_mb,modified
0,amphibian_tree,Directory,,0.00,2025-09-18 06:28:53
1,bird_tree,Directory,,0.00,2025-09-18 06:29:04
2,mammal_tree,Directory,,0.00,2025-09-18 06:29:17
3,ray-finned fishes_tree,Directory,,0.00,2025-09-18 06:28:16
4,rays_species.nwk,File,.nwk,0.00,2025-09-18 07:20:57
5,rays_tree,Directory,,0.00,2025-09-18 07:21:52
6,shark_tree,Directory,,0.00,2025-09-18 06:28:30
7,squamate_tree,Directory,,0.00,2025-09-18 06:28:41
8,vertlife_taxonomies.csv,File,.csv,1.35,2025-09-18 01:08:30



📂 Directory: amphibian_tree
------------------------------------------------------------


,name,type,extension,size_mb,modified
9,amph_shl_new_Consensus_7238.tre,File,.tre,0.28,2025-09-18 06:07:46
10,taxonomy_amphibians.csv,File,.csv,0.61,2025-09-18 01:33:22



📂 Directory: bird_tree
------------------------------------------------------------


...


📂 Directory: squamate_tree
------------------------------------------------------------


,name,type,extension,size_mb,modified
22,sqamate_names.csv,File,.csv,0.19,2025-09-18 05:57:01



📄 ALL FILES SUMMARY (Files only, top 30):


,relative_path,extension,size_mb,modified
4,rays_species.nwk,.nwk,0.00,2025-09-18 07:20:57
8,vertlife_taxonomies.csv,.csv,1.35,2025-09-18 01:08:30
9,amphibian_tree/amph_shl_new_Consensu...,.tre,0.28,2025-09-18 06:07:46
10,amphibian_tree/taxonomy_amphibians.csv,.csv,0.61,2025-09-18 01:33:22
11,bird_tree/Aves_species.nwk,.nwk,0.35,2025-09-18 07:22:23
12,bird_tree/Aves_taxonomy.csv,.csv,2.27,2025-09-18 06:00:57
13,mammal_tree/MamPhy_fullPosterior_BDv...,.trees,1766.59,2025-09-18 05:42:41
14,ray-finned fishes_tree/1105_protein_...,.tre,0.05,2025-09-18 06:25:44
15,ray-finned fishes_tree/12862_2017_95...,.tre,0.12,2025-09-18 05:07:41
16,ray-finned fishes_tree/12862_2017_95...,.csv,0.14,2025-09-18 05:08:16



💾 File listing stored in 'files_df' variable for further analysis
   Use files_df to access the complete file inventory


## Cell 5 – File Filtering and Search
## Optional: Filter Files by Criteria
Utility functions to filter and search through the discovered files.

In [ ]:
# ===== Cell 5 =====
# Optional file filtering and search utilities

def filter_files_by_extension(df, extensions):
    """
    Filter files by file extension(s).

    Args:
        df (DataFrame): Files DataFrame from list_directory_contents
        extensions (str or list): Extension(s) to filter by (e.g., '.csv', ['.txt', '.csv'])

    Returns:
        DataFrame: Filtered DataFrame
    """
    if isinstance(extensions, str):
        extensions = [extensions]

    extensions = [ext.lower() if ext.startswith('.') else f'.{ext.lower()}' for ext in extensions]
    return df[df['extension'].isin(extensions)]

def search_files_by_name(df, search_term, case_sensitive=False):
    """
    Search files by name pattern.

    Args:
        df (DataFrame): Files DataFrame from list_directory_contents
        search_term (str): Term to search for in filenames
        case_sensitive (bool): Whether search should be case sensitive

    Returns:
        DataFrame: Filtered DataFrame
    """
    if case_sensitive:
        mask = df['name'].str.contains(search_term, na=False)
    else:
        mask = df['name'].str.contains(search_term, case=False, na=False)

    return df[mask]

# Example usage (uncomment to use):
# print("🔍 Example: Show only CSV files")
# csv_files = filter_files_by_extension(files_df, '.csv')
# display(csv_files)

# print("\n🔍 Example: Search for files containing 'phylo'")
# phylo_files = search_files_by_name(files_df, 'phylo')
# display(phylo_files)

### Cell 2: Comprehensive Taxonomic Matrix Implementation Workflow (Markdown)
This markdown cell contains your strategic plan and documentation. It should be kept as is.


#### Comprehensive Taxonomic Matrix Implementation Workflow

## 🎯 **Strategic Implementation Plan**

### **Phase 1: Complete VertLife Matrix (Recommended First)**
Build complete matrix of everything in VertLife, then add custom data as needed.

**Advantages:**
- ✅ **Computational Efficiency**: Process all VertLife data once
- ✅ **Future-Proof**: Handles new species without reprocessing
- ✅ **Comprehensive MRCA**: All possible ancestor relationships pre-calculated
- ✅ **Clade Cluster Analysis**: Complete phylogenetic context for clustering

### **Phase 2: Custom Data Integration**
Add teleost fish and other missing groups using Newick trees.

### **Phase 3: UniProt Species Mapping**
Map your stream.gz species to the comprehensive matrix.

## 📁 ** Directory Organization**

Keep `_SHARED_DATA/` clean by organizing all taxonomy-related materials under `TAXONOMY/`:

```
_SHARED_DATA/
└── TAXONOMY/                                       # All taxonomy materials
    ├── VERTLIFE_DATA/                             # Raw VertLife downloads
    │   ├── mammaltree/
    │   │   ├── taxonomy_mamPhy_5911species.csv
    │   │   └── Completed_5911sp_topoCons_NDexp.zip
    │   ├── birdtree/
    │   │   ├── [bird_taxonomy_file].csv
    │   │   └── [bird_tree_file].zip
    │   ├── amphibiantree/
    │   │   ├── [amphibian_taxonomy_file].csv
    │   │   └── [amphibian_tree_file].zip
    │   ├── squamatetree/
    │   │   ├── [squamate_taxonomy_file].csv
    │   │   └── [squamate_tree_file].zip
    │   ├── sharktree/
    │   │   ├── [shark_taxonomy_file].csv
    │   │   └── [shark_tree_file].zip
    │   └── vertlife_taxonomies.csv               # Master taxonomy file
    │
    ├── CUSTOM_TREES/                              # Custom phylogenetic data
    │   ├── teleost_fish_tree.nwk                 # Newick format tree
    │   ├── teleost_fish_taxonomy.csv             # Teleost taxonomy mapping
    │   ├── additional_vertebrate_trees/          # Other custom trees
    │   └── tree_metadata.json                    # Custom tree documentation
    │
    ├── TAXONOMY_MAPS/                             # ID mapping & corrections
    │   ├── ncbi_taxonomy.csv                     # NCBI taxonomy database
    │   ├── taxdump/                              # NCBI taxdump files
    │   │   ├── names.dmp
    │   │   └── nodes.dmp
    │   ├── species_name_corrections.csv          # Manual name corrections
    │   ├── uniprot_to_vertlife_mapping.csv       # Species mapping results
    │   └── taxon_id_lookup.json                  # Taxon ID relationships
    │
    ├── PROCESSING/                                # Temporary processing files
    │   ├── logs/                                 # Processing logs
    │   ├── intermediate/                         # Temp matrices
    │   └── validation/                           # QC reports
    │
    └── OUTPUT/                                    # Final matrices & reports
        ├── comprehensive_taxonomic_matrix_latest.json
        ├── vertebrate_temporal_matrix_latest.json
        ├── clade_cluster_analysis.json
        ├── taxonomic_matrix_report.txt
        ├── unmapped_species_report.txt
        └── processing_statistics.json
```


## 🔧 **Implementation Steps**

### **Step 1: Data Preparation**

#### **Download VertLife Data**
```bash
# Priority order for your collagen analysis:
# 1. Mammals (densest coverage)
curl -O https://data.vertlife.org/mammaltree/taxonomy_mamPhy_5911species.csv
curl -O https://data.vertlife.org/mammaltree/Completed_5911sp_topoCons_NDexp.zip

# 2. Birds (significant collagen diversity)
curl -O https://data.vertlife.org/birdtree/[bird_taxonomy_file]
curl -O https://data.vertlife.org/birdtree/[bird_tree_file]

# 3. Other vertebrates
# Download amphibiantree, squamatetree, sharktree as available
```

#### **Prepare Custom Trees**
```bash
# For teleost fish (no VertLife yet)
# Use your existing Newick tree + create taxonomy file
cp existing_teleost_tree.nwk CUSTOM_TREES/teleost_fish_tree.nwk
# Create teleost_fish_taxonomy.csv manually or from existing data
```

#### **NCBI Taxonomy Integration**
```bash
# Download NCBI taxonomy for taxon ID matching
wget ftp://ftp.ncbi.nih.gov/pub/taxonomy/taxdump.tar.gz
# Extract names.dmp and nodes.dmp for taxon ID mapping
```

### **Step 2: Matrix Building**

```python
# Run the comprehensive matrix builder
from comprehensive_taxonomic_matrix_builder import build_comprehensive_taxonomic_matrix

matrix_file, cluster_analysis = build_comprehensive_taxonomic_matrix(
    uniprot_stream_path=Path("uniprot_vertebrata.stream.gz"),
    config_dir=Path("_SHARED_DATA"),
    output_dir=Path("_SHARED_DATA/TAXONOMY")
)
```

### **Step 3: Integration with ExonMapper**

```python
# In ExonMapper Cell 11 (enhanced):
def load_comprehensive_taxonomic_matrix():
    matrix_path = Path(SHARED_DATA_DIR) / "TAXONOMY" / "comprehensive_taxonomic_matrix_latest.json"
    
    if matrix_path.exists():
        with open(matrix_path) as f:
            matrix_data = json.load(f)
        
        authoritative_clade_map = matrix_data['species_matrix']
        clade_clusters = matrix_data['clade_clusters']
        
        log(f"✅ Comprehensive taxonomic matrix loaded", "INFO")
        log(f"   Species mapped: {len(authoritative_clade_map):,}", "INFO")
        log(f"   Clade clusters: {len(clade_clusters)}", "INFO")
        
        return authoritative_clade_map, clade_clusters
    
    return {}, {}
```

## 🧬 **MRCA & Temporal Calculations**

### **Most Recent Common Ancestor (MRCA) Data**
Each species entry will contain:

```json
{
  "Homo_sapiens": {
    "vertebrate_group": "mammals",
    "vertlife_species": "homo_sapiens",
    "taxon_id": 9606,
    "clade": "great_apes",
    "time_to_root_ma": 95.7,
    "taxonomic_path": [
      {
        "node_name": "hominidae_ancestor",
        "branch_length_ma": 7.2,
        "distance_to_root_ma": 88.5,
        "is_internal": true
      },
      {
        "node_name": "catarrhini_ancestor",
        "branch_length_ma": 18.3,
        "distance_to_root_ma": 70.2,
        "is_internal": true
      }
    ],
    "mrca_calculations": {
      "vs_pan_troglodytes": {"ancestor": "hominidae_ancestor", "age_ma": 7.2},
      "vs_macaca_mulatta": {"ancestor": "catarrhini_ancestor", "age_ma": 25.5}
    }
  }
}
```

## 🔍 **Clade Cluster Analysis**

### **Primate Cluster (Cell 30 Implementation)**
```python
def build_primate_cluster():
    primate_species = clade_clusters['great_apes'] + clade_clusters['old_world_monkeys']
    
    # Temporal coherence analysis
    primate_ages = [species_matrix[sp]['time_to_root_ma'] for sp in primate_species]
    
    cluster_stats = {
        'species_count': len(primate_species),
        'mean_age_ma': np.mean(primate_ages),
        'coherence_score': calculate_cluster_coherence(primate_ages),
        'recommended_for_scaffold': coherence_score > 0.8
    }
    
    return primate_cluster, cluster_stats
```

### **Progressive Cluster Building**
1. **Primate cluster** (Cell 30) - highest priority, best annotation
2. **Rodent cluster** - high species count, good coverage  
3. **Ungulate cluster** - important for collagen diversity
4. **Carnivore cluster** - moderate size, good phylogenetic distance
5. **Other mammalian clusters** - fill remaining gaps

## 🌡️ **Water Temperature & Speciation Integration**

### **Environmental Context Data**
```python
# Add environmental context to species matrix
environmental_factors = {
    "water_temperature_preference": "warm/cold/variable",
    "habitat_type": "marine/freshwater/terrestrial",
    "speciation_rate_context": "high/medium/low",
    "collagen_evolution_pressure": "temperature_adaptation/mechanical_stress"
}

# Integration with MRCA data for collagen analysis
def analyze_temperature_speciation_correlation(species_matrix):
    # Correlate rapid speciation (short branch lengths) with warm water habitats
    # Link to collagen thermal stability requirements
    pass
```

## 📊 **Unmapped Species Handling**

### **Species Name Correction Dictionary**
```python
# Create mapping for common naming variations
name_corrections = {
    "Sus scrofa domesticus": "Sus_scrofa",
    "Bos taurus taurus": "Bos_taurus",
    "Mus musculus musculus": "Mus_musculus",
    # Add corrections based on unmapped species report
}
```

### **Manual Review Process**
1. **Generate unmapped species report**
2. **Identify naming variations** (subspecies, synonyms)
3. **Create correction dictionary**
4. **Re-run mapping with corrections**
5. **Flag remaining unmapped for biological review**

## 🔄 **Processing Efficiency Strategy**

### **Computational Approach**
```python
# Most efficient workflow:
def build_matrix_efficiently():
    # 1. Load ALL VertLife trees once
    all_trees = load_all_vertlife_datasets()
    
    # 2. Pre-calculate ALL pairwise MRCA relationships  
    # (computationally intensive but done once)
    mrca_matrix = calculate_comprehensive_mrca_matrix(all_trees)
    
    # 3. Extract UniProt species list
    uniprot_species = extract_species_from_stream()
    
    # 4. Map UniProt species to VertLife (fast lookup)
    species_mapping = map_species_to_trees(uniprot_species, all_trees)
    
    # 5. Extract relevant MRCA data (fast subset)
    relevant_mrca = subset_mrca_matrix(mrca_matrix, species_mapping)
    
    return comprehensive_matrix
```

## 🎯 **Success Metrics**

### **Matrix Quality Indicators**
- **Species Coverage**: >85% of UniProt vertebrates mapped
- **Temporal Precision**: ±0.5 Ma accuracy for MRCA calculations  
- **Clade Coherence**: >0.8 coherence score for major clades
- **MRCA Completeness**: All pairwise relationships calculated

### **Integration Success**
- **ExonMapper Speed**: <30 seconds to load matrix (vs hours of processing)
- **Cluster Quality**: Clear separation of major vertebrate clades
- **Manual Review**: <100 species requiring name corrections

This approach gives you the **complete taxonomic foundation** needed for your collagen phylogenetic analysis while maintaining production reliability and computational efficiency.

## Cell 10: The Main Python Script (Class and Functions)
This cell contains your entire `CleanTaxonomicMatrixBuilder` class and the main `build_clean_taxonomic_matrix` function. The command-line parsing part at the end has been removed.

In [ ]:
# ===== Clean Taxonomy Matrix Builder - New Directory Structure =====
# Updated for clean organization under _SHARED_DATA/TAXONOMY/

import pandas as pd
import polars as pl
import numpy as np
from pathlib import Path
import json
import gzip
import zipfile
import argparse
from typing import Dict, List, Tuple, Optional, Set
from collections import defaultdict, Counter
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from ete3 import Tree

class CleanTaxonomicMatrixBuilder:
    """
    Comprehensive taxonomic matrix builder with clean directory organization.
    All taxonomy materials organized under _SHARED_DATA/TAXONOMY/
    """

    def __init__(self, shared_data_dir: Path):
        self.shared_data_dir = Path(shared_data_dir)

        # Clean directory structure under TAXONOMY/
        self.taxonomy_base = self.shared_data_dir / "TAXONOMY"

        # Organized subdirectories
        self.vertlife_data_dir = self.taxonomy_base / "VERTLIFE_DATA"
        self.custom_trees_dir = self.taxonomy_base / "CUSTOM_TREES"
        self.taxonomy_maps_dir = self.taxonomy_base / "TAXONOMY_MAPS"
        self.processing_dir = self.taxonomy_base / "PROCESSING"
        self.output_dir = self.taxonomy_base / "OUTPUT"

        # Create directory structure
        self._create_directory_structure()

        # Data storage
        self.vertebrate_groups = {
            'mammals': {'tree': None, 'taxonomy': None, 'loaded': False},
            'birds': {'tree': None, 'taxonomy': None, 'loaded': False},
            'amphibians': {'tree': None, 'taxonomy': None, 'loaded': False},
            'squamates': {'tree': None, 'taxonomy': None, 'loaded': False},
            'sharks': {'tree': None, 'taxonomy': None, 'loaded': False},
            'teleost_fish': {'tree': None, 'taxonomy': None, 'loaded': False}
        }

        # Processing data
        self.ncbi_taxonomy = None
        self.species_to_taxon_id = {}
        self.comprehensive_matrix = {}
        self.processing_stats = defaultdict(int)
        self.unmapped_species = []
        self.clade_clusters = defaultdict(list)

        # Setup logging
        self.log_file = self.processing_dir / "logs" / f"matrix_building_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

    def _create_directory_structure(self):
        """Create clean directory structure."""
        directories = [
            self.taxonomy_base,
            self.vertlife_data_dir,
            self.custom_trees_dir,
            self.taxonomy_maps_dir,
            self.processing_dir,
            self.output_dir,
            self.processing_dir / "logs",
            self.processing_dir / "intermediate",
            self.processing_dir / "validation"
        ]

        # VertLife subdirectories
        for group in ['mammaltree', 'birdtree', 'amphibiantree', 'squamatetree', 'sharktree']:
            directories.append(self.vertlife_data_dir / group)

        for directory in directories:
            directory.mkdir(parents=True, exist_ok=True)

        print(f"📁 Created clean directory structure under {self.taxonomy_base}")

    def log(self, message: str, level: str = "INFO"):
        """Enhanced logging with file output."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_message = f"[{timestamp}] {level}: {message}"

        print(log_message)

        # Also log to file
        self.log_file.parent.mkdir(parents=True, exist_ok=True)
        with open(self.log_file, 'a') as f:
            f.write(log_message + "\n")

    def setup_data_sources(self) -> Dict[str, bool]:
        """
        Setup and validate all data sources in clean directory structure.
        """
        self.log("Setting up data sources in clean directory structure")

        setup_status = {}

        # Check VertLife data availability
        vertlife_configs = {
            'mammals': {
                'taxonomy': 'mammaltree/taxonomy_mamPhy_5911species.csv',
                'tree': 'mammaltree/Completed_5911sp_topoCons_NDexp.zip',
                'priority': 1
            },
            'birds': {
                'taxonomy': 'birdtree/taxonomy_birds.csv',
                'tree': 'birdtree/birdtree_complete.zip',
                'priority': 2
            },
            'amphibians': {
                'taxonomy': 'amphibiantree/taxonomy_amphibians.csv',
                'tree': 'amphibiantree/amphibian_tree.zip',
                'priority': 3
            },
            'squamates': {
                'taxonomy': 'squamatetree/taxonomy_squamates.csv',
                'tree': 'squamatetree/squamate_tree.zip',
                'priority': 4
            },
            'sharks': {
                'taxonomy': 'sharktree/taxonomy_sharks.csv',
                'tree': 'sharktree/shark_tree.zip',
                'priority': 5
            }
        }

        self.log("Checking VertLife data availability:")
        for group, config in vertlife_configs.items():
            taxonomy_path = self.vertlife_data_dir / config['taxonomy']
            tree_path = self.vertlife_data_dir / config['tree']

            if taxonomy_path.exists() and tree_path.exists():
                setup_status[group] = True
                self.log(f"  ✅ {group}: Data files found")
            else:
                setup_status[group] = False
                self.log(f"  ❌ {group}: Missing files")
                self.log(f"     Expected: {taxonomy_path}")
                self.log(f"     Expected: {tree_path}")

        # Check custom trees
        custom_configs = {
            'teleost_fish': {
                'taxonomy': 'teleost_fish_taxonomy.csv',
                'tree': 'teleost_fish_tree.nwk'
            }
        }

        self.log("Checking custom tree data:")
        for group, config in custom_configs.items():
            taxonomy_path = self.custom_trees_dir / config['taxonomy']
            tree_path = self.custom_trees_dir / config['tree']

            if taxonomy_path.exists() and tree_path.exists():
                setup_status[group] = True
                self.log(f"  ✅ {group}: Custom data found")
            else:
                setup_status[group] = False
                self.log(f"  ⚠️ {group}: Custom data not found (optional)")

        # Check NCBI taxonomy
        ncbi_files = [
            self.taxonomy_maps_dir / "ncbi_taxonomy.csv",
            self.taxonomy_maps_dir / "taxdump" / "names.dmp"
        ]

        ncbi_available = any(f.exists() for f in ncbi_files)
        setup_status['ncbi_taxonomy'] = ncbi_available
        self.log(f"  {'✅' if ncbi_available else '⚠️'} NCBI taxonomy: {'Available' if ncbi_available else 'Not found (will use name matching)'}")

        return setup_status
    def load_ncbi_taxonomy_data(self):
        """Load NCBI taxonomy data for taxon ID mapping."""
        self.log("Loading NCBI taxonomy data...")

        # Check if PhASTM TaxonomyManager is available
        if 'taxonomy_manager' in globals():
            try:
                tax_manager = globals()['taxonomy_manager']
                if hasattr(tax_manager, 'database_ready') and tax_manager.database_ready:
                    self.log("✅ Using PhASTM TaxonomyManager")
                    self.ncbi_taxonomy = tax_manager
                    return
            except Exception as e:
                self.log(f"⚠️ PhASTM TaxonomyManager error: {e}")

        # Check for local NCBI files
        ncbi_files = [
            self.taxonomy_maps_dir / "taxdump" / "names.dmp",
            self.taxonomy_maps_dir / "taxdump" / "nodes.dmp"
        ]

        if all(f.exists() for f in ncbi_files):
            self.log("✅ Found local NCBI taxonomy files")
            # Could implement ETE3 loading here
        else:
            self.log("ℹ️ No NCBI taxonomy found - using name matching")

    def load_vertebrate_datasets(self) -> Dict[str, bool]:
        """Load all vertebrate datasets (VertLife and custom)."""
        self.log("Loading vertebrate datasets...")

        results = {}

        # Check your existing tree directories
        existing_dirs = {
            'ray_finned_fish': self.shared_data_dir.parent / 'ray-finned fishes_tree',
            'mammals': self.shared_data_dir.parent / 'mammal_tree',
            'birds': self.shared_data_dir.parent / 'bird_tree',
            'amphibians': self.shared_data_dir.parent / 'amphibian_tree',
            'squamates': self.shared_data_dir.parent / 'squamate_tree',
            'sharks': self.shared_data_dir.parent / 'shark_tree'
        }

        for group, dir_path in existing_dirs.items():
            if dir_path.exists():
                self.log(f"✅ Found {group} data at {dir_path}")
                results[group] = True
                self.vertebrate_data[group] = {'path': dir_path, 'loaded': True}
            else:
                self.log(f"⚠️ {group} data not found")
                results[group] = False

        return results

    def extract_uniprot_species_from_stream(self, uniprot_stream_path: Path) -> Set[str]:
        """Extract unique species names from UniProt stream.gz file."""
        self.log(f"Extracting species from UniProt stream: {uniprot_stream_path}")

        if not uniprot_stream_path.exists():
            self.log(f"❌ Stream file not found: {uniprot_stream_path}")
            return set()

        species_set = set()

        try:
            import gzip
            import pandas as pd

            self.log("📖 Reading UniProt stream file...")
            with gzip.open(uniprot_stream_path, 'rt') as f:
                # Sample the file first to understand structure
                df = pd.read_csv(f, sep='\t', low_memory=False, nrows=10000)

                # Extract species from organism_name column
                if 'organism_name' in df.columns:
                    unique_species = df['organism_name'].dropna().unique()
                    species_set.update(unique_species)
                    self.log(f"✅ Extracted {len(species_set)} species from organism_name")

                # Also check lineage if available
                if 'lineage' in df.columns:
                    for lineage in df['lineage'].dropna():
                        if isinstance(lineage, str) and ';' in lineage:
                            species = lineage.split(';')[-1].strip()
                            if species:
                                species_set.add(species)

                    self.log(f"✅ Total species after lineage: {len(species_set)}")

        except Exception as e:
            self.log(f"❌ Error reading stream: {e}")
            return set()

        return species_set

    def build_comprehensive_species_matrix(self, uniprot_species: Set[str]) -> Dict:
        """Build the comprehensive taxonomic matrix."""
        self.log("Building comprehensive species matrix...")

        matrix = {}
        processed = 0

        # Process each species (limit for initial testing)
        for species in list(uniprot_species)[:500]:  # Process first 500 for testing
            entry = {
                'species_name': species,
                'source': 'uniprot_collagen_stream',
                'timestamp': datetime.now().isoformat(),
                'phylogenetic_assignments': {},
                'taxonomy_info': {}
            }

            # Try to get taxonomy info if available
            if hasattr(self, 'ncbi_taxonomy') and self.ncbi_taxonomy:
                try:
                    tax_info = self.ncbi_taxonomy.get_taxonomy_info_by_name(species)
                    if tax_info:
                        entry['taxonomy_info'] = tax_info
                except:
                    pass

            matrix[species] = entry
            processed += 1

            if processed % 100 == 0:
                self.log(f"Processed {processed} species...")

        self.log(f"✅ Matrix built for {processed} species")
        return matrix

    def save_comprehensive_matrix(self) -> Tuple[Path, Path]:
        """Save the comprehensive matrix and reports."""
        self.log("Saving comprehensive matrix and reports...")

        # Ensure output directory exists
        self.output_dir.mkdir(parents=True, exist_ok=True)

        # File paths
        matrix_file = self.output_dir / "phastm_taxonomic_matrix_latest.json"
        report_file = self.output_dir / "phastm_taxonomic_report.txt"

        # Save matrix as JSON
        import json
        with open(matrix_file, 'w') as f:
            json.dump(self.species_matrix, f, indent=2, default=str)

        # Save processing report
        with open(report_file, 'w') as f:
            f.write("PhASTM Taxonomic Matrix Processing Report\n")
            f.write("=" * 50 + "\n")
            f.write(f"Generated: {datetime.now()}\n")
            f.write(f"Total species in matrix: {len(self.species_matrix)}\n")
            f.write(f"Unmapped species: {len(self.unmapped_species)}\n")
            f.write("\nProcessing Statistics:\n")
            for key, value in self.processing_stats.items():
                f.write(f"  {key}: {value}\n")

        self.log(f"✅ Matrix saved: {matrix_file}")
        self.log(f"✅ Report saved: {report_file}")

        return matrix_file, report_file
    def _save_unmapped_species_report(self, unmapped_file: Path):
        """Save unmapped species for manual review."""
        with open(unmapped_file, 'w') as f:
            f.write("UNMAPPED SPECIES REPORT\n")
            f.write("=" * 30 + "\n\n")
            f.write(f"Total unmapped: {len(self.unmapped_species)}\n\n")
            f.write("Species requiring manual review:\n")

            for i, species in enumerate(self.unmapped_species, 1):
                f.write(f"{i:4d}. {species}\n")

        self.log(f"📝 Unmapped species report saved: {unmapped_file.name}")


# ===== MAIN EXECUTION FUNCTION =====

def build_clean_taxonomic_matrix(
    shared_data_dir: Path,
    uniprot_stream_path: Path
) -> Tuple[Path, Dict]:
    """
    Build comprehensive taxonomic matrix using clean directory structure.

    Args:
        shared_data_dir: Path to _SHARED_DATA directory
        uniprot_stream_path: Path to UniProt stream.gz file

    Returns:
        Tuple of (matrix_file_path, processing_stats)
    """
    print("🧬 Building Comprehensive Taxonomic Matrix with Clean Directory Structure")
    print(f"📁 Shared data directory: {shared_data_dir}")
    print(f"📥 UniProt stream: {uniprot_stream_path}")

    # Create builder with clean directory structure
    builder = CleanTaxonomicMatrixBuilder(shared_data_dir)

    try:
        # Step 1: Setup and validate data sources
        print("\n" + "="*60)
        setup_status = builder.setup_data_sources()

        # Step 2: Load NCBI taxonomy for taxon ID mapping
        print("\n" + "="*60)
        builder.load_ncbi_taxonomy_data()

        # Step 3: Load vertebrate datasets
        print("\n" + "="*60)
        loading_results = builder.load_vertebrate_datasets()

        if not any(loading_results.values()):
            print("❌ No vertebrate datasets loaded successfully")
            return None, {}

        # Step 4: Extract UniProt species
        print("\n" + "="*60)
        uniprot_species = builder.extract_uniprot_species_from_stream(uniprot_stream_path)

        if not uniprot_species:
            print("❌ No species extracted from UniProt stream")
            return None, {}

        # Step 5: Build comprehensive species matrix
        print("\n" + "="*60)
        species_matrix = builder.build_comprehensive_species_matrix(uniprot_species)

        # Step 6: Save results to clean OUTPUT directory
        print("\n" + "="*60)
        matrix_file, report_file = builder.save_comprehensive_matrix()

        print(f"\n✅ CLEAN TAXONOMIC MATRIX COMPLETE!")
        print(f"📁 Clean directory structure: {builder.taxonomy_base}")
        print(f"📊 Species processed: {len(uniprot_species):,}")
        print(f"🔗 Successfully mapped: {len(species_matrix):,}")
        print(f"💾 Matrix file: {matrix_file}")
        print(f"📈 Report file: {report_file}")

        return matrix_file, builder.processing_stats

    except Exception as e:
        print(f"❌ Matrix building failed: {e}")
        import traceback
        traceback.print_exc()
        return None, {}


## Cell 11: Execute the Matrix Builder
This is the new "runner" cell. **You must update the placeholder paths in this cell.** After you run Cell 3, this cell will call the main function with your specific file paths and start the process.

In [ ]:
# ===== Cell 11: Superior Data Structure Matrix Builder Execution =====
# Updated to work with your SUPERIOR data organization (master taxonomy + selected trees)

from pathlib import Path

# ==============================================================================
# === 🌟 SUPERIOR DATA STRUCTURE INTEGRATION                                ===
# ==============================================================================

# --- Step 1: Use your actual file paths ---
SHARED_DATA_DIRECTORY = Path("/content/drive/MyDrive/Colab_Notebooks/GitHub/_SHARED_DATA")
UNIPROT_STREAM_FILE = Path("/content/drive/MyDrive/Colab_Notebooks/GitHub/_SHARED_DATA/Uniprot/collagens/stream.gz")

print("🌟 Superior Data Structure Taxonomic Matrix Builder")
print(f"📁 Shared data directory: {SHARED_DATA_DIRECTORY}")
print(f"📥 UniProt stream: {UNIPROT_STREAM_FILE}")

# --- Step 2: Verify file existence ---
if not SHARED_DATA_DIRECTORY.exists():
    print(f"❌ Shared data directory not found: {SHARED_DATA_DIRECTORY}")
    print("Please update the path to point to your actual _SHARED_DATA location")
else:
    print(f"✅ Shared data directory found")

if not UNIPROT_STREAM_FILE.exists():
    print(f"❌ UniProt stream file not found: {UNIPROT_STREAM_FILE}")
    print("Please verify the UniProt stream file location")
else:
    print(f"✅ UniProt stream file found ({UNIPROT_STREAM_FILE.stat().st_size / (1024*1024):.1f} MB)")

# --- Step 3: Check for your superior data files ---
VERTLIFE_DATA_DIR = SHARED_DATA_DIRECTORY / "TAXONOMY" / "VERTLIFE_DATA"
MASTER_TAXONOMY_FILE = VERTLIFE_DATA_DIR / "vertlife_taxonomies.csv"

print(f"\n🔍 Checking your superior data structure:")
print(f"📁 VertLife data directory: {VERTLIFE_DATA_DIR}")

if MASTER_TAXONOMY_FILE.exists():
    size_mb = MASTER_TAXONOMY_FILE.stat().st_size / (1024*1024)
    print(f"✅ Master taxonomy file found: vertlife_taxonomies.csv ({size_mb:.2f} MB)")
else:
    print(f"❌ Master taxonomy file not found: {MASTER_TAXONOMY_FILE}")

# Check for your superior tree files
tree_check_patterns = [
    "mammal_tree/MamPhy_*.trees",
    "bird_tree/Aves_*.nwk",
    "bird_tree/Aves_*.csv",
    "amphibian_tree/*.tre",
    "amphibian_tree/*.csv",
    "shark_tree/*.nex",
    "shark_tree/*.csv",
    "ray-finned fishes_tree/*.nwk",
    "squamate_tree/*.csv"
]

print(f"\n📊 Superior phylogenetic data inventory:")
total_tree_size = 0
found_groups = []

for pattern in tree_check_patterns:
    search_path = VERTLIFE_DATA_DIR / pattern
    if search_path.parent.exists():
        matches = list(search_path.parent.glob(search_path.name))
        if matches:
            group = pattern.split('/')[0].replace('_tree', '').replace(' fishes', '_fish')
            group_size = sum(f.stat().st_size for f in matches) / (1024*1024)
            total_tree_size += group_size
            found_groups.append(group)
            print(f"  ✅ {group}: {len(matches)} files ({group_size:.2f} MB)")

print(f"📊 Total superior data: {total_tree_size:.1f} MB across {len(found_groups)} vertebrate groups")

# --- Step 4: Run the superior matrix builder ---
try:
    print("\n" + "="*70)
    print("🚀 Starting Superior Data Structure Matrix Builder")
    print("="*70)

    # Create the builder (uses the superior CleanTaxonomicMatrixBuilder from Cell 3)
    builder = CleanTaxonomicMatrixBuilder(SHARED_DATA_DIRECTORY)

    # Step 1: Setup data sources (will find your superior data automatically)
    print("\n🔍 Setting up superior data sources...")
    setup_status = builder.setup_data_sources()

    if not setup_status.get('master_taxonomy'):
        print("❌ Master taxonomy not found - check file location")
        raise FileNotFoundError("Master taxonomy file required")

    # Step 2: Load NCBI taxonomy (PhASTM integration if available)
    print("\n📚 Loading taxonomy data...")
    builder.load_ncbi_taxonomy_data()

    # Step 3: Load vertebrate datasets (your superior trees)
    print("\n🌳 Loading superior phylogenetic datasets...")
    loading_results = builder.load_vertebrate_datasets()

    loaded_groups = sum(1 for k, v in loading_results.items() if v and k != 'master_taxonomy')
    print(f"✅ Successfully loaded {loaded_groups} superior phylogenetic datasets")

    # Step 4: Extract UniProt species (PhASTM collagen data)
    print("\n🧬 Extracting species from UniProt collagen stream...")
    uniprot_species = builder.extract_uniprot_species_from_stream(UNIPROT_STREAM_FILE)

    if not uniprot_species:
        print("❌ No species extracted from UniProt stream")
        raise ValueError("UniProt stream processing failed")

    print(f"✅ Extracted {len(uniprot_species)} unique species from collagen data")

    # Step 5: Build comprehensive species matrix (using superior data)
    print("\n🏗️ Building comprehensive species matrix with superior data...")
    species_matrix = builder.build_comprehensive_species_matrix(uniprot_species)
    builder.species_matrix = species_matrix

    if not species_matrix:
        print("❌ Matrix building failed")
        raise ValueError("Matrix building returned empty result")

    print(f"✅ Built comprehensive matrix for {len(species_matrix)} species")

    # Step 6: Save results
    print("\n💾 Saving superior taxonomic matrix...")
    matrix_file, report_file = builder.save_comprehensive_matrix()

    # === SUCCESS SUMMARY ===
    print("\n" + "="*70)
    print("🎉 SUCCESS! SUPERIOR DATA STRUCTURE MATRIX COMPLETE!")
    print("="*70)

    print(f"\n📊 Processing Summary:")
    print(f"   🧬 Species processed: {len(species_matrix):,}")
    print(f"   🌳 Phylogenetic groups: {len(found_groups)}")
    print(f"   📈 Total data processed: {total_tree_size:.1f} MB")
    print(f"   🎯 Master taxonomy species: {len(builder.master_taxonomy_df) if hasattr(builder, 'master_taxonomy_df') and builder.master_taxonomy_df is not None else 'N/A'}")

    print(f"\n💾 Output Files:")
    print(f"   📋 Matrix: {matrix_file}")
    print(f"   📄 Report: {report_file}")

    print(f"\n🔗 PhASTM Ecosystem Integration:")
    print(f"   ✅ Ready for ExonMapper integration")
    print(f"   ✅ Ready for Brain ecosystem tools")
    print(f"   ✅ Compatible with ZooMS workflows")

    print(f"\n🌟 Data Quality Advantages:")
    print(f"   ✅ Comprehensive master taxonomy (vs fragmented files)")
    print(f"   ✅ Superior selected phylogenetic trees")
    print(f"   ✅ Includes advanced teleost fish data")
    print(f"   ✅ Research-grade Bayesian phylogenies")
    print(f"   ✅ PhASTM ecosystem optimized")

    # Quick validation
    if matrix_file.exists() and matrix_file.stat().st_size > 1000:
        print(f"\n✅ VALIDATION PASSED: Matrix file created successfully")

        # Show sample of what was created
        try:
            with open(matrix_file, 'r') as f:
                import json
                sample_data = json.load(f)
                sample_species = list(sample_data.keys())[:3]
                print(f"\n📋 Sample species in matrix: {', '.join(sample_species)}")
        except:
            pass
    else:
        print(f"\n⚠️ Matrix file may be incomplete")

    print(f"\n🎯 NEXT STEPS:")
    print(f"   1. Matrix is ready for integration with other PhASTM tools")
    print(f"   2. Use matrix_file path in ExonMapper or other components")
    print(f"   3. Superior data structure provides comprehensive phylogenetic context")

except Exception as e:
    print(f"\n❌ ERROR: {e}")
    import traceback
    print("\n🔍 Full error traceback:")
    traceback.print_exc()

    print(f"\n🛠️ TROUBLESHOOTING:")
    print(f"   1. Ensure vertlife_taxonomies.csv is in VERTLIFE_DATA/")
    print(f"   2. Verify your phylogenetic tree files are accessible")
    print(f"   3. Check that Cell 3 has been updated with superior data structure code")
    print(f"   4. Confirm UniProt stream file is readable")

🌟 Superior Data Structure Taxonomic Matrix Builder
📁 Shared data directory: /content/drive/MyDrive/Colab_Notebooks/GitHub/_SHARED_DATA
📥 UniProt stream: /content/drive/MyDrive/Colab_Notebooks/GitHub/_SHARED_DATA/Uniprot/collagens/stream.gz
✅ Shared data directory found
✅ UniProt stream file found (124.8 MB)

🔍 Checking your superior data structure:
📁 VertLife data directory: /content/drive/MyDrive/Colab_Notebooks/GitHub/_SHARED_DATA/TAXONOMY/VERTLIFE_DATA
✅ Master taxonomy file found: vertlife_taxonomies.csv (1.35 MB)

📊 Superior phylogenetic data inventory:
  ✅ mammal: 1 files (1766.59 MB)
  ✅ bird: 1 files (0.35 MB)
  ✅ bird: 1 files (2.27 MB)
  ✅ amphibian: 1 files (0.28 MB)
  ✅ amphibian: 1 files (0.61 MB)
  ✅ shark: 1 files (12.00 MB)
  ✅ shark: 1 files (0.02 MB)
  ✅ ray-finned_fish: 1 files (0.76 MB)
  ✅ squamate: 1 files (0.19 MB)
📊 Total superior data: 1783.1 MB across 9 vertebrate groups

🚀 Starting Superior Data Structure Matrix Builder
📁 Created clean directory structure und